# DM2022 ISA5810 Lab2 Homework

This file is run on the Kaggle platform.

## Install Required Package
I use ktrain to build & train the ML model.

In [1]:
!pip install ktrain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.3/25.3 MB 41.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 50.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 263.7/263.7 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 72.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.8/468.8 kB 28.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Preparing metadata (setup.py) ... - done
  Preparing metadata (setup.py) ... - done
  Preparing metadata (setup.py) ... - done
  Preparing metadata (setup.py) ... - done
  Preparing metadata (setup.py) ... - done
  Preparing metadata (setup.py) ... - done
  Created wheel for ktrain: filename=ktrain-0.31.10-py3-none-any.whl size=25312982 sha256=b2516b4dd5dc6b8a59f2fe07fe98096cf9bcfe4b2af03cc0ba01a2cb4

In [2]:
import ktrain
from ktrain import text

## Data Preprocess

Extract data and split the training data into training and validation dataset.

In [ ]:
import json
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
# extract the data we need
data = []
with open('../input/dm2022-isa5810-lab2-homework/tweets_DM.json') as f:
    for line in f:
        tweet = json.loads(line)['_source']['tweet']
        data.append((tweet['tweet_id'], tweet['text']))

df = pd.DataFrame.from_records(data, columns=['tweet_id', 'text'])

In [ ]:
# merge with emotion and identification
df_emotion = pd.read_csv('../input/dm2022-isa5810-lab2-homework/emotion.csv')
df_identification = pd.read_csv(
    '../input/dm2022-isa5810-lab2-homework/data_identification.csv')
df = df.merge(df_identification, how="outer")
df = df.merge(df_emotion, how="outer")

In [ ]:
df_train = df[df['identification'] == 'train']
df_test = df[df['identification'] == 'test']

# 80% for training and others for validation
train, validate = train_test_split(df_train, test_size=0.2)
train.to_csv('./train.csv', index=False)
validate.to_csv('./validate.csv', index=False)
df_test.to_csv('./test.csv', index=False)

train.csv, validate.csv, test.csv will be uploaded to Kaggle input.

## Data Preparation

In [3]:
# load the data we need
import pandas as pd
df_train = pd.read_csv('../input/dm2022lab2preprocess/train.csv', lineterminator='\n')
df_validate = pd.read_csv('../input/dm2022lab2preprocess/validate.csv', lineterminator='\n')
df_test = pd.read_csv('../input/dm2022lab2preprocess/test.csv', lineterminator='\n')
len(df_train), len(df_validate)

(1164450, 291113)

## Train

In [6]:
# use bert model
model_name = 'distilbert-base-uncased'
categories = emotions = df_train.emotion.unique()
trans = text.Transformer(model_name, maxlen=32, class_names=categories)

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [7]:
# set up our train_data and val_data by using the transformer
train_data = trans.preprocess_train(list(df_train['text']), list(df_train['emotion']))
val_data = trans.preprocess_train(list(df_validate['text']), list(df_validate['emotion']))

preprocessing train...
language: en
train sequence lengths:
	mean : 15
	95percentile : 25
	99percentile : 28


/opt/conda/lib/python3.7/site-packages/ktrain/utils.py:738: UserWarning: class_names argument was ignored, as they were extracted from string labels in dataset
  "class_names argument was ignored, as they were extracted from string labels in dataset"


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Is Multi-Label? False
preprocessing train...
language: en
train sequence lengths:
	mean : 15
	95percentile : 25
	99percentile : 28


Is Multi-Label? False


In [8]:
# define the parameters for training
model = trans.get_classifier()
learner = ktrain.get_learner(model, train_data=train_data, val_data=val_data, batch_size=16)

Downloading:   0%|          | 0.00/347M [00:00<?, ?B/s]

In [10]:
# set the learning rate = 2e-5 and run for 5 epoch
learner.fit_onecycle(2e-5, 5)



begin training using onecycle policy with max lr of 2e-05...
Epoch 1/5
3639/3639 [==============================] - 254s 63ms/step - loss: 1.5396 - accuracy: 0.4431 - val_loss: 1.3905 - val_accuracy: 0.4955
Epoch 2/5
3639/3639 [==============================] - 239s 62ms/step - loss: 1.2618 - accuracy: 0.5516 - val_loss: 1.2673 - val_accuracy: 0.5429
Epoch 3/5
3639/3639 [==============================] - 241s 63ms/step - loss: 1.0811 - accuracy: 0.6176 - val_loss: 1.2532 - val_accuracy: 0.5516
Epoch 4/5
3639/3639 [==============================] - 241s 63ms/step - loss: 0.8195 - accuracy: 0.7135 - val_loss: 1.3551 - val_accuracy: 0.5451
Epoch 5/5
3639/3639 [==============================] - 241s 63ms/step - loss: 0.5787 - accuracy: 0.8055 - val_loss: 1.5256 - val_accuracy: 0.5424


In [11]:
# view the result
learner.validate()

              precision    recall  f1-score   support

           0       0.40      0.23      0.30       424
           1       0.63      0.59      0.61      2644
           2       0.43      0.40      0.41      1432
           3       0.51      0.42      0.46       632
           4       0.60      0.70      0.65      5001
           5       0.44      0.52      0.48      1880
           6       0.33      0.21      0.25       453
           7       0.49      0.40      0.44      2090

    accuracy                           0.54     14556
   macro avg       0.48      0.43      0.45     14556
weighted avg       0.54      0.54      0.54     14556



array([[  99,   24,   84,   10,   75,  111,    6,   15],
       [  11, 1565,   90,   50,  563,  142,   19,  204],
       [  47,   61,  567,   31,  251,  395,   40,   40],
       [   8,   66,   42,  268,  135,   72,   14,   27],
       [  29,  424,  147,   75, 3491,  299,   52,  484],
       [  33,   92,  285,   42,  331,  983,   37,   77],
       [   5,   34,   49,   24,  138,   89,   93,   21],
       [  13,  223,   63,   28,  788,  123,   23,  829]])

## Prediction

In [12]:
# get predictor and save the model
predictor = ktrain.get_predictor(learner.model, preproc=trans)
predictor.save('model')


In [16]:
# predict
df_test['emotion'] = df_test['text'].apply(lambda x: predictor.predict(x))

In [17]:
# save the output
df_test.to_csv('./output.csv', columns=['tweet_id', 'emotion'], header=['id','emotion'], index=False)